Experimental implementation of the gap statistic.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
from joblib import Parallel, delayed # for parallelization
from itertools import product

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Clustering
from stepmix.stepmix import StepMix
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering, HDBSCAN
from scipy.spatial.distance import mahalanobis

# Evaluation
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.neighbors import BallTree
import torch
from torchmetrics.clustering import DunnIndex
from collections import Counter
from kneed import KneeLocator

# Visualization
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull

# Preparation
## Data

In [ ]:
data2004_i = pd.read_parquet("data/data2004_i.parquet") # load imputed data

# Dataset with numeric outcomes
data_n = data2004_i[[
    # Q2
    'clseusa_n', # 'clsetown_n', 'clsestat_n', 'clsenoam_n',
    # Q3
    'ambornin_n', 'amcit_n', 'amlived_n', 'amenglsh_n', 
    'amchrstn_n', 'amgovt_n', 'amfeel_n', # 'amancstr_n',
    # Q4
    'amcitizn_n', 'amshamed_n', 'belikeus_n', 'ambetter_n', 'ifwrong_n', # 'amsports_n', 'lessprd_n',
    # Q5
    'proudsss_n', 'proudgrp_n', 'proudpol_n', 'prouddem_n', 'proudeco_n',
    'proudspt_n', 'proudart_n', 'proudhis_n', 'proudmil_n', 'proudsci_n']]

# Dataset with categorical outcomes
data_f = data2004_i[[
    # Q2
    'clseusa_f', # 'clsetown_f', 'clsestat_f', 'clsenoam_f',
    # Q3
    'ambornin_f', 'amcit_f', 'amlived_f', 'amenglsh_f', 
    'amchrstn_f', 'amgovt_f', 'amfeel_f', # 'amancstr_f',
    # Q4
    'amcitizn_f', 'amshamed_f', 'belikeus_f', 'ambetter_f', 'ifwrong_f', # 'amsports_f', 'lessprd_f',
    # Q5
    'proudsss_f', 'proudgrp_f', 'proudpol_f', 'prouddem_f', 'proudeco_f',
    'proudspt_f', 'proudart_f', 'proudhis_f', 'proudmil_f', 'proudsci_f']]

# Dataset with controls
controls = data2004_i[[
    'sex', 'race_f', 'born_usa', 'party_fs', 'religstr_f', 
    'reltrad_f', 'region_f']]

## Parameters

In [ ]:
max_clust = 5
max_threads = 8

CVI = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn', 'inertia']

## Validity indexes

In [ ]:
# Custom score functions to avoid throwing errors when undefined
def sil_score(data, pred_clust):
    try: 
        sil_score = silhouette_score(data, pred_clust)
    except ValueError:
        sil_score = np.nan
    return sil_score

def ch_score(data, pred_clust):
    try:
        ch_score = calinski_harabasz_score(data, pred_clust)
    except ValueError:
        ch_score = np.nan
    return ch_score

def db_score(data, pred_clust):
    try:
        db_score = davies_bouldin_score(data, pred_clust)
    except ValueError:
        db_score = np.nan
    return db_score

def dunn_score(data, pred_clust):
    torch_data = np.array(data)
    torch_data = torch.tensor(torch_data, dtype=torch.float32)
    torch_pred_clust = torch.tensor(pred_clust, dtype=torch.int64)

    dunn_metric = DunnIndex()
    
    try:
        dunn_score = float(dunn_metric(torch_data, torch_pred_clust))
    except Exception:
        dunn_score = np.nan
 
    return dunn_score

def inertia(data, pred_clust):
    data = np.asarray(data)
    
    inertia = 0
    for cluster in np.unique(pred_clust):
        cluster_points = data[pred_clust == cluster]
        cluster_centroid = np.mean(cluster_points, axis=0)
        inertia += np.sum((cluster_points - cluster_centroid) ** 2)
        
    return inertia

def clust_size(pred_clust):
    cluster_sizes = Counter(pred_clust)
    min_size = min(cluster_sizes.values())
    max_size = max(cluster_sizes.values())
    
    return min_size, max_size

In [ ]:
# Function to return all validity indexes at once
def get_metrics(model, params, n, data, pred_clust, **additional_metrics):
    base_metrics = {
        'model': model,
        'params': params,
        'n_clust': n,
        'min_clust_size': clust_size(pred_clust)[0],
        'max_clust_size': clust_size(pred_clust)[1],
        'silhouette': float(sil_score(data, pred_clust)),
        'calinski_harabasz': float(ch_score(data, pred_clust)),
        'davies_bouldin': float(db_score(data, pred_clust)),
        'dunn': float(dunn_score(data, pred_clust)),
        'inertia': float(inertia(data, pred_clust))
    }

    base_metrics.update(additional_metrics)
    return base_metrics

# Models

In [ ]:
# Parameters
clust_range = range(1, max_clust+1)

opt_params = {
    'method': 'gradient',
    'intercept': True,
    'max_iter': 2500,
}

In [ ]:
# Fit models without covariates
def do_StepMix(data, n, msrt, covar):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)

        latent_mod = StepMix(
            n_components = n, 
            measurement = msrt, 
            n_init = 3,
            init_params = 'kmeans',
            structural_params = opt_params,
            random_state = 123,
            progress_bar = 0)
        
        latent_mod.fit(data)
        pred_clust = latent_mod.predict(data)

        model = 'latent'
        params = f"msrt = {msrt}, covar = {covar}"
        loglik = latent_mod.score(data)
        aic = latent_mod.aic(data)
        bic = latent_mod.aic(data)
        entropy = latent_mod.entropy(data)

    return get_metrics(model, params, n, data, pred_clust, LL = loglik, aic = aic, bic = bic, entropy = entropy)

data = data_f.apply(lambda col: LabelEncoder().fit_transform(col))
cat_results = Parallel(n_jobs=8)(delayed(do_StepMix)(data, n, 'categorical', 'without') for n in clust_range)

num_results = Parallel(n_jobs=8)(delayed(do_StepMix)(data_n, n, 'continuous', 'without') for n in clust_range)

latent_all = pd.concat([pd.DataFrame(cat_results), pd.DataFrame(num_results)])

In [ ]:
class FlexibleKMeans:
    """
    K-Means implementation supporting different distance metrics and center computation methods.
    
    Parameters:
    -----------
    n_clusters : int
        Number of clusters
    metric : str, default='euclidean'
        Distance metric: 'euclidean', 'manhattan', 'chebyshev'
    center_method : str, default='mean'
        Method to compute cluster centers: 'mean', 'median', 'medoid'
    max_iter : int, default=100
        Maximum number of iterations
    n_init : int, default=10
        Number of times the k-means algorithm will be run with different centroid seeds.
        The final result will be the best output of n_init consecutive runs in terms of inertia.
    random_state : int or None, default=None
        Random state for reproducibility
    """
    
    def __init__(self, n_clusters, metric='euclidean', center_method='mean', 
                 max_iter=100, n_init=10, random_state=None):
        self.n_clusters = n_clusters
        self.metric = metric
        self.center_method = center_method
        self.max_iter = max_iter
        self.n_init = n_init
        self.random_state = random_state
        
        # Define mapping from user-friendly names to scipy metrics
        self.metric_mapping = {
            'euclidean': 'euclidean',
            'manhattan': 'cityblock',
            'chebyshev': 'chebyshev'
        }
        
        # Validate inputs
        valid_metrics = list(self.metric_mapping.keys())
        if metric not in valid_metrics:
            raise ValueError(f"metric must be one of {valid_metrics}")
            
        valid_centers = ['mean', 'median', 'medoid']
        if center_method not in valid_centers:
            raise ValueError(f"center_method must be one of {valid_centers}")
            
        if self.n_init <= 0:
            raise ValueError("n_init should be > 0")
    
    def _compute_distances(self, X, centers):
        """Compute distances between points and centers using specified metric."""
        return cdist(X, centers, metric=self.metric_mapping[self.metric])
    
    def _compute_centers(self, X, labels):
        """Compute new centers using specified method."""
        new_centers = np.zeros((self.n_clusters, X.shape[1]))
        
        for i in range(self.n_clusters):
            cluster_points = X[labels == i]
            
            if len(cluster_points) == 0:
                continue
                
            if self.center_method == 'mean':
                new_centers[i] = np.mean(cluster_points, axis=0)
            
            elif self.center_method == 'median':
                new_centers[i] = np.median(cluster_points, axis=0)
            
            elif self.center_method == 'medoid':
                # For medoid, find the point that minimizes sum of distances to other points
                distances = self._compute_distances(cluster_points, cluster_points)
                medoid_idx = np.argmin(np.sum(distances, axis=1))
                new_centers[i] = cluster_points[medoid_idx]
        
        return new_centers
    
    def _single_fit(self, X, seed):
        """Perform a single run of k-means with given random seed."""
        if seed is not None:
            np.random.seed(seed)
            
        # Initialize centers randomly
        idx = np.random.choice(len(X), self.n_clusters, replace=False)
        centers = X[idx].copy()
        
        for iteration in range(self.max_iter):
            # Store old centers for convergence check
            old_centers = centers.copy()
            
            # Assign points to nearest center
            distances = self._compute_distances(X, centers)
            labels = np.argmin(distances, axis=1)
            
            # Update centers
            centers = self._compute_centers(X, labels)
            
            # Check for convergence
            if np.allclose(old_centers, centers):
                n_iter = iteration + 1
                break
        else:
            n_iter = self.max_iter
            
        # Compute final inertia
        final_distances = self._compute_distances(X, centers)
        inertia = np.sum(np.min(final_distances, axis=1) ** 2)
        
        return centers, labels, inertia, n_iter
    
    def fit(self, X):
        """Fit the model to the data."""
        # Convert pandas DataFrame to numpy array if necessary
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        X = np.asarray(X)
        
        # Initialize best solution tracking
        best_inertia = np.inf
        best_labels = None
        best_centers = None
        best_n_iter = None
        
        # Run k-means n_init times
        for init in range(self.n_init):
            # Generate seed for this initialization
            if self.random_state is not None:
                seed = self.random_state + init
            else:
                seed = None
                
            # Perform single k-means run
            centers, labels, inertia, n_iter = self._single_fit(X, seed)
            
            # Update best solution if current one is better
            if inertia < best_inertia:
                best_centers = centers
                best_labels = labels
                best_inertia = inertia
                best_n_iter = n_iter
        
        # Store best solution
        self.cluster_centers_ = best_centers
        self.labels_ = best_labels
        self.inertia_ = best_inertia
        self.n_iter_ = best_n_iter
        
        return self
    
    def fit_predict(self, X):
        """Fit the model and return cluster labels."""
        return self.fit(X).labels_
    
    def predict(self, X):
        """Predict the closest cluster for each sample in X."""
        # Convert pandas DataFrame to numpy array if necessary
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        X = np.asarray(X)
        
        distances = self._compute_distances(X, self.cluster_centers_)
        return np.argmin(distances, axis=1)

In [ ]:
def do_kmeans(data, n, dist, link):
    kmeans = FlexibleKMeans(
        n_clusters = n,
        metric = dist,
        center_method = link,
        n_init = 15)

    pred_clust = kmeans.fit_predict(data)
    
    model = 'kmeans'
    params = f"dist = {dist}, link = {link}"
    
    return get_metrics(model, params, n, data, pred_clust)

distances = ['euclidean', 'manhattan', 'chebyshev']
linkages = ['mean', 'median', 'medoid']
kmeans_params = product(distances, linkages)

clust_range = range(1, max_clust+1)
kmeans_params_range = product(clust_range, kmeans_params)

results = Parallel(n_jobs=max_threads)(delayed(do_kmeans)(data_n, n, dist, link) for n, (dist, link) in kmeans_params_range)
kmeans_all = pd.DataFrame(results)

In [ ]:
def do_AHC(data, n, dist, link):
    ahc = AgglomerativeClustering(
        n_clusters = n,
        metric = dist,
        linkage = link)
    
    ahc.fit(data)
    pred_clust = ahc.labels_

    model = 'AHC'
    params = f"dist = {dist}, link = {link}"

    return get_metrics(model, params, n, data, pred_clust)

distances = ['manhattan', 'euclidean', 'chebyshev', 'hamming']
linkages = ['single', 'average', 'complete']
ahc_params = [*product(distances, linkages), ('euclidean', 'ward')]

clust_range = range(1, max_clust+1)
ahc_params_range = product(clust_range, ahc_params)

results = Parallel(n_jobs=max_threads)(delayed(do_AHC)(data_n, n, dist, link) for n, (dist, link) in ahc_params_range)
ahc_all = pd.DataFrame(results)

In [ ]:
all_models = pd.concat([latent_all, kmeans_all, ahc_all]).reset_index(drop=True)

In [ ]:
all_models

# Gap stat

Adapted from: https://www.geeksforgeeks.org/gap-statistics-for-optimal-number-of-cluster/

In [ ]:
def dict_to_strg(d):
    return ', '.join(f"{key} = {value}" for key, value in d.items())

# Generate reference data from a uniform distribution
def gen_ref_data(data):
    return np.random.uniform(low=data.min(axis=0), 
                            high=data.max(axis=0), 
                            size=data.shape)

# Create empty df to store results
def create_empty_df(indices):
    cols = ['model', 'params', 'n_clust'] + \
       [f'{index}_gs' for index in CVI] + \
       [f'{index}_s' for index in CVI]
    
    df = pd.DataFrame(columns=cols)

    float_cols = [col for col in cols if col not in ['model', 'params', 'n_clust']]
    df[float_cols] = df[float_cols].astype('float64')
    
    df['model'] = df['model'].astype('object')
    df['params'] = df['params'].astype('object')
    df['n_clust'] = df['n_clust'].astype('int64')

    return df

In [ ]:
# Compute the Gap Statistic
def compute_gap_statistic(data, model, params, iters):   

    str_params = dict_to_strg(params)
    gap_values = create_empty_df(CVI)

    if model == 'latent': 
        n_min = 1
    else: 
        n_min = 2

    # Loop over n values
    for n in range(n_min, max_clust+1):
    
        # Fit the model on random datasets
        rand_scores_all = pd.DataFrame()
        
        for _ in range(iters):
            rand_data = gen_ref_data(data)
            
            if model == 'latent':
                if params.get('covar') == 'without':
                    rand_scores = do_StepMix(rand_data, n, **params)
                else:
                    return None

            elif model == 'kmeans':
                rand_scores = do_kmeans(rand_data, n, **params)

            elif model == 'AHC':
                rand_scores = do_AHC(rand_data, n, **params)
            
            rand_scores = pd.DataFrame([rand_scores])
            rand_scores_all = pd.concat([rand_scores_all, rand_scores], ignore_index=True)

        # Retrive scores for the assessed model
        mod_scores = all_models.loc[(all_models['model'] == model) & 
                                    (all_models['params'] == str_params) & 
                                    (all_models['n_clust'] == n)]

        # Calculate the Gap statistic and s value for each validity index
        for index in CVI:
            rand_ind = rand_scores_all[index]
            mod_ind = mod_scores[index]

            # Rescale the Silhouette index on [0,1] to avoid errors when it is negative
            if index == 'silhouette':
                rand_ind = (rand_ind + 1) / 2
                mod_ind = (mod_ind + 1) / 2
                
            gap = np.log(np.mean(rand_ind)) - np.log(mod_ind)
            s = np.std(np.log(rand_ind)) * np.sqrt(1 + (1 / iters))

            # Store the results
            ## Check if the corresponding row exists in the df
            row_id = ((gap_values['model'] == model) & 
                      (gap_values['params'] == str_params) & 
                      (gap_values['n_clust'] == n))

            if gap_values[row_id].empty:
            ## If not, create a new one
                new_row = {
                    'model': model,
                    'params': str_params,
                    'n_clust': n,
                    f'{index}_gs': gap.values[0],
                    f'{index}_s': s
                }
                new_row = pd.DataFrame([new_row])
                gap_values = pd.concat([gap_values, new_row], ignore_index=True)
            
            else:
            # Otherwise, update the existing row
                gap_values.loc[row_id, f'{index}_gs'] = gap.values[0]
                gap_values.loc[row_id, f'{index}_s'] = s

    return gap_values

In [ ]:
iters = 3
model = 'latent'
params = {'msrt': 'continuous', 'covar': 'without'}

gap_values = compute_gap_statistic(data_n, model, params, iters)

In [ ]:
gap_values

In [ ]:
iters = 3
model = 'AHC'
params = {'dist': 'manhattan', 'link': 'single'}

gap_values = compute_gap_statistic(data_n, model, params, iters)
gap_values

In [ ]:
iters = 3
model = 'kmeans'
params = {'dist': 'manhattan', 'link': 'mean'}

gap_values = compute_gap_statistic(data_n, model, params, iters)
gap_values

In [ ]:
params = {'dist': 'manhattan', 'link': 'medoid'}
do_kmeans(data_n, 3, **params)

In [ ]:
params = {'dist': 'manhattan', 'link': 'single'}
do_AHC(data_n, 3, **params)

In [ ]:
# Define parameters grid

models = ['latent', 'kmeans', 'AHC']

msrt = ['categorical', 'continuous']
covar = ['without', 'with']
latent_params = list(product(msrt, covar))

dist = ['euclidean', 'manhattan', 'chebyshev']
link = ['mean', 'median', 'medoid']
kmeans_params = list(product(dist, link))

dist = ['manhattan', 'euclidean', 'chebyshev', 'hamming']
link = ['single', 'average', 'complete']
ahc_params = [*product(dist, link), ('euclidean', 'ward')]

params = {
    'latent': latent_params,
    'kmeans': kmeans_params,
    'AHC': ahc_params
}

param_names = {
    'latent': ['msrt', 'covar'],
    'kmeans': ['dist', 'link'],
    'AHC': ['dist', 'link']
}

grid = [
    (model, dict(zip(param_names[model], param_values)))
    for model in models
    for param_values in params[model]
]

In [ ]:
results = Parallel(n_jobs=max_threads)(
    delayed(compute_gap_statistic)(data_n, iters=5, model=model, params=config)
    for model, config in grid
)
gap_values = pd.concat(results).reset_index(drop=True)

In [ ]:
# Check if the number of rows is consistent with all_models

gap_values

In [ ]:
# Select the optimal number of clusters
def get_best_gap(model, params, index):
    # Subset gap_values to the right model and params 
    rows_id = ((gap_values['model'] == model) & (gap_values['params'] == params))
    df = gap_values[rows_id]

    # Extract gap and s values
    gap = df[f'{index}_gs']
    s = df[f'{index}_s']

    # Define cluster range (adjusting for index-based calculations)
    n_min = df['n_clust'].min() - 1 # Start at -1 to align with zero-based indexing
    n_max = df['n_clust'].max() - 2 # Reduce range to avoid out-of-bounds errors
    
    stats = []
    
    # Select rows such that GS(k) >= GS(k+1) - s(k+1)
    for i in range(n_min, n_max):
        stat = gap[i] - gap[i+1] + s[i+1]
        if stat >= 0: 
            stats.append([i+1, stat])

    # Return optimal cluster number
    stats = np.array(stats)
    if stats.size == 0:
        best_n = 'none'
    else:
        best_n = int(stats[np.argmin(stats[:, 1]), 0])

    return best_n

In [ ]:
best_gap = pd.DataFrame()
new_row = pd.DataFrame([{'model': model, 'params': params}])
best_gap = pd.concat([best_gap, new_row], ignore_index=True)

for index in CVI:
    best_n = get_best_gap(model, params, index)
    best_gap[index] = best_n

best_gap

In [ ]:
# Create df to store results
cols = ['model', 'params', 'n_clust'] + \
       [f'{index}_abs' for index in CVI] + \
       [f'{index}_elbow' for index in CVI] + \
       [f'{index}_gap' for index in CVI]

candidate_models = pd.DataFrame(columns=cols)

float_cols = [col for col in cols if col not in ['model', 'params', 'n_clust']]
candidate_models[float_cols] = candidate_models[float_cols].astype('float64')

candidate_models['model'] = candidate_models['model'].astype('object')
candidate_models['params'] = candidate_models['params'].astype('object')
candidate_models['n_clust'] = candidate_models['n_clust'].astype('int64')

In [ ]:
# Find best n
# no need to parallelize as the operations are quite simple
for model in models:
    for config in params[model]:
        for index in CVI:
            best_n = get_best_gap(model, params, index)

            row_id = ((candidate_models['model'] == model) & 
                      (candidate_models['params'] == params) &
                      (candidate_models['n_clust'] == best_n))
            
            # Check if the corresponding row exists in the df
            if candidate_models[row_id].empty:
                new_row = {
                    'model': model,
                    'params': params,
                    'n_clust': best_n,
                    f'{index}_gap': 1
                }
                new_row = pd.DataFrame([new_row])
                candidate_models = pd.concat([candidate_models, new_row], ignore_index=True)

            # Otherwise, update the existing row
            else:
                candidate_models.loc[row_id, f'{index}_gap'] = 1

In [ ]:
# Compute the Gap Statistic
def compute_gap_statistic(data, n_min, n_max, iters, model, params):
    gap_values = create_empty_df(CVI)
    
    # Loop over n values
    for n in range(n_min, n_max + 1):
    
        # Fit the model on random datasets
        rand_scores_all = pd.DataFrame()
        
        for _ in range(iters):
            rand_data = gen_ref_data(data)
            
            if model == 'LPA' and params == 'without covariates':
                rand_scores = do_StepMix(n, 'continuous', rand_data)

            elif model == 'kmeans':
                config = extract_params(params)
                rand_scores = do_kmeans(rand_data, n, **config)

            elif model == 'AHC':
                config = extract_params(params)
                rand_scores = do_AHC(rand_data, n, **config)
            
            rand_scores = pd.DataFrame([rand_scores])
            rand_scores_all = pd.concat([rand_scores_all, rand_scores], ignore_index=True)

        # Retrive scores for the assessed model
        mod_scores = all_models.loc[(all_models['model'] == model) & 
                                     (all_models['params'] == params) & 
                                     (all_models['n_clust'] == n)]

        # Calculate the Gap statistic and s value for each validity index
        for index in CVI:
            rand_ind = rand_scores_all[index]
            mod_ind = mod_scores[index]

            # Rescale the Silhouette index on [0,1] to avoid errors when it is negative
            if index == 'silhouette':
                rand_ind = (rand_ind + 1) / 2
                mod_ind = (mod_ind + 1) / 2
                
            gap = np.log(np.mean(rand_ind)) - np.log(mod_ind)
            s = np.std(np.log(rand_ind)) * np.sqrt(1 + (1 / iters))

            # Store the results
            ## Check if the corresponding row exists in the df
            row_id = ((gap_values['model'] == model) & 
                      (gap_values['params'] == params) & 
                      (gap_values['n_clust'] == n))

            if gap_values[row_id].empty:
            ## If not, create a new one
                new_row = {
                    'model': model,
                    'params': params,
                    'n_clust': n,
                    f'{index}_gs': gap.values[0],
                    f'{index}_s': s
                }
                new_row = pd.DataFrame([new_row])
                gap_values = pd.concat([gap_values, new_row], ignore_index=True)
            
            else:
            # Otherwise, update the existing row
                gap_values.loc[row_id, f'{index}_gs'] = gap.values[0]
                gap_values.loc[row_id, f'{index}_s'] = s

    return gap_values